In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('example.db')

In [3]:
cursor = conn.cursor()

In [4]:
create_table = '''CREATE TABLE employees(id INTEGER PRIMARY KEY, name TEXT, position TEXT, salary REAL)'''
cursor.execute(create_table)

In [5]:
insert_rows = '''INSERT INTO employees(name, position, salary) VALUES ('Rajath', 'Thought Leader', 5000)'''
cursor.execute(insert_rows)

In [7]:
conn.commit()

In [10]:
query = '''SELECT * FROM employees'''
cursor.execute(query)
data = cursor.fetchall()
print(data)

[(1, 'Rajath', 'Thought Leader', 5000.0)]


In [11]:
update_row = """UPDATE employees SET salary = 5500 WHERE name = 'Rajath' """
cursor.execute(update_row)
conn.commit()

In [12]:
employee_data = ('Elon Musk', 'Developer', 6000)
cursor.execute("INSERT INTO employees (name, position, salary) VALUES (?, ?, ?)", employee_data)
conn.commit()

In [13]:
cursor.execute("""DELETE FROM employees WHERE name = 'Rajath' """)
conn.commit()

In [14]:
conn.close()

In [1]:
import mysql.connector
from mysql.connector import Error

In [32]:
def create_server_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection


In [33]:
connection = create_server_connection('192.168.0.116', 'rajath', 'rajathkumar', 'sampledb')

MySQL Database connection successful


In [6]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [7]:
create_table_query = """
CREATE TABLE employees (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100),
    position VARCHAR(100),
    salary DECIMAL(10, 2)
);
"""

# Assuming you already have a 'connection' object as created earlier
execute_query(connection, create_table_query)

Query successful


In [8]:
def create_record(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [9]:
create_query = """
INSERT INTO employees (name, position, salary) 
VALUES ('Rajath', 'Manager', 5000);
"""
create_record(connection, create_query)


Query successful


In [10]:
create_query = """
INSERT INTO employees (name, position, salary) 
VALUES ('Musk', 'Engineer', 6000);
"""
create_record(connection, create_query)


Query successful


In [40]:
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

In [35]:
query = """SELECT * FROM employees"""
results = read_query(connection, query)
for result in results:
    print(result)

(1, 'Rajath', 'Manager', Decimal('5000.00'))
(2, 'Musk', 'Engineer', Decimal('6000.00'))


In [42]:
query = """SELECT * FROM employees WHERE name = 'Rajath' """
results = read_query(connection, query)
print(results)

[(1, 'Rajath', 'Manager', Decimal('5000.00'))]


In [43]:
def update_record(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")


In [45]:
update_query = """
UPDATE employees
SET salary = 5500
WHERE name = 'Rajath';
"""
update_record(connection, update_query)


Query successful


In [46]:
def delete_record(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [47]:
delete_query = "DELETE FROM employees WHERE name = 'Rajath'"
delete_record(connection, delete_query)


Query successful


In [48]:
connection.close()

In [50]:
from sqlmodel import SQLModel, Field

In [51]:
class Employee(SQLModel, table=True):
    id: int = Field(default=None, primary_key=True)
    name: str
    position: str
    salary: float

In [52]:
from sqlmodel import create_engine

sqlite_file_name = "database.db"
sqlite_url = f"sqlite:///{sqlite_file_name}"

engine = create_engine(sqlite_url)

SQLModel.metadata.create_all(engine)

In [64]:
from sqlmodel import Session

def create_employee(employee: Employee):
    with Session(engine) as session:
        session.add(employee)
        session.commit()
        session.refresh(employee)
        return employee
    
new_employee = Employee(name="Rajath", position="Engineer", salary=5000)
created_employee = create_employee(new_employee)

In [55]:
def get_employee(employee_id: int):
    with Session(engine) as session:
        employee = session.get(Employee, employee_id)
        return employee

# Usage
employee = get_employee(1)

In [56]:
employee

Employee(salary=5000.0, id=1, position='Engineer', name='Rajath')

In [65]:
from sqlmodel import select

def get_all_employees():
    with Session(engine) as session:
        return list(session.exec(select(Employee)))

# Usage
employees = get_all_employees()
# print(employees)

In [66]:
employees

[Employee(salary=5000.0, id=1, position='Engineer', name='Rajath')]

In [62]:
def update_employee_salary(employee_id: int, new_salary: float):
    with Session(engine) as session:
        employee = session.get(Employee, employee_id)
        if employee:
            employee.salary = new_salary
            session.add(employee)
            session.commit()
            session.refresh(employee)
        return employee

# Usage
update_employee_salary(1, 55000)

Employee(salary=55000.0, id=1, position='Engineer', name='Rajath')

In [63]:
def delete_employee(employee_id: int):
    with Session(engine) as session:
        employee = session.get(Employee, employee_id)
        if employee:
            session.delete(employee)
            session.commit()

# Usage
delete_employee(1)

In [68]:
from pymongo import MongoClient

# MongoDB connection string
connection_string = "mongodb://192.168.0.116:27017/"

# Create a MongoClient
client = MongoClient(connection_string)

# Connect to the database
db = client['example_db']

In [69]:
employees = db.employees

In [70]:
def insert_employee(employee_data):
    return employees.insert_one(employee_data).inserted_id

# Usage
new_employee = {
    "name": "Rajath",
    "position": "Manager",
    "salary": 5000
}
insert_employee(new_employee)

ObjectId('65ea49ea18031aed18aae9f9')

In [71]:
def find_employee(name):
    return employees.find_one({"name": name})

# Usage
employee = find_employee("Rajath")
print(employee)

{'_id': ObjectId('65ea49ea18031aed18aae9f9'), 'name': 'Rajath', 'position': 'Manager', 'salary': 5000}


In [72]:
def find_all_employees():
    return list(employees.find())

# Usage
all_employees = find_all_employees()
print(all_employees)

[{'_id': ObjectId('65ea49ea18031aed18aae9f9'), 'name': 'Rajath', 'position': 'Manager', 'salary': 5000}]


In [73]:
def update_employee_salary(name, new_salary):
    return employees.update_one({"name": name}, {"$set": {"salary": new_salary}})

# Usage
update_employee_salary("Rajath", 5500)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [74]:
def delete_employee(name):
    return employees.delete_one({"name": name})

# Usage
delete_employee("Rajath")

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

In [75]:
client.close()

In [79]:
import requests

url = 'https://broadridge.free.beeceptor.com'
response = requests.get(url)

print(response.text)  # prints the response content



Hey ya! Great to see you here. Btw, nothing is configured for this request path. Create a rule and start building a mock API.



In [80]:
payload = {'key1': 'value1', 'key2': 'value2'}
response = requests.get(url, params=payload)

print(response.url)  # prints the full URL with parameters


https://broadridge.free.beeceptor.com/?key1=value1&key2=value2


In [82]:
url = 'https://broadridge.free.beeceptor.com/hello'
data = {'key': 'broadridge'}

response = requests.post(url, data=data)

print(response.text)


{"Hello":"Broadridge"}


In [83]:
json_data = {'key': 'value'}
response = requests.post(url, json=json_data)

print(response.text)


{"Hello":"Broadridge"}


In [86]:
url = 'https://broadridge.free.beeceptor.com/hello'
data = {'my': 'name'}

response = requests.put(url, data=data)

print(response.text)


{"status":"aws"}


In [87]:
import requests
from bs4 import BeautifulSoup


In [88]:
url = 'http://amazon.in'
response = requests.get(url)


In [89]:
soup = BeautifulSoup(response.content, 'lxml')

In [91]:
title = soup.title.text
print(title)


Online Shopping site in India: Shop Online for Mobiles, Books, Watches, Shoes and More - Amazon.in


In [92]:
first_div = soup.find('div')
print(first_div)


<div id="a-page"><script data-a-state='{"key":"a-wlab-states"}' type="a-state">{"AUI_TNR_V2_180836":"C","AUI_PRELOAD_261698":"C","AUI_72554":"C","AUI_A11Y_4_835613":"C","AUI_KILLSWITCH_CSA_LOGGER_372963":"C","AUI_A11Y_SR_678508":"C","AUI_PCI_RISK_BANNER_210084":"C","AUI_A11Y_6_837773":"C","AUI_TEMPLATE_WEBLAB_CACHE_333406":"C","AUI_A11Y_1_699934":"C","AUI_AMZN_IMG_861115":"C","AUI_REL_NOREFERRER_NOOPENER_309527":"C","AUI_AMZN_IMG_GATE_861116":"C"}</script><script>typeof uex === 'function' && uex('ld', 'portal-bb', {wb: 1})</script><!-- sp:end-feature:start-body -->
<!-- sp:feature:csm:body-open -->
<!-- sp:end-feature:csm:body-open -->
<!-- sp:feature:nav-inline-js -->
<!-- NAVYAAN JS -->
<script type="text/javascript">!function(n){function e(n,e){return{m:n,a:function(n){return[].slice.call(n)}(e)}}document.createElement("header");var r=function(n){function u(n,r,u){n[u]=function(){a._replay.push(r.concat(e(u,arguments)))}}var a={};return a._sourceName=n,a._replay=[],a.getNow=function

In [96]:
links = soup.find_all('a')
for link in links:
    print(link.get('href'))


None
None
/ref=nav_logo
None
/customer-preferences/edit?ie=UTF8&preferencesReturnUrl=%2F&ref_=topnav_lang
https://www.amazon.in/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.in%2F%3F_encoding%3DUTF8%26ref_%3Dnav_ya_signin&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=inflex&openid.mode=checkid_setup&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0
/gp/css/order-history?ref_=nav_orders_first
https://www.amazon.in/gp/cart/view.html?ref_=nav_cart
/gp/site-directory?ref_=nav_em_js_disabled
/fresh?ref_=nav_cs_fresh
/minitv?ref_=nav_avod_desktop_topnav
/b/32702023031?node=32702023031&ld=AZINSOANavDesktop_T3&ref_=nav_cs_sell_T3
/gp/bestsellers/?ref_=nav_cs_bestsellers
/mobile-phones/b/?ie=UTF8&node=1389401031&ref_=nav_cs_mobiles
/deals?ref_=nav_cs_gb
/electronics/b/?ie=UTF8&node=976419031&ref_=nav_cs_electronics
/prime?ref_=